# Step-Wise Backtesting of RSI Strategy on XAUUSDm

This notebook performs a step-wise backtest of the RSI strategy on `XAUUSDm` data using Backtesting.py. We'll load the data, compute the RSI indicator, run the backtest, and visualize the results.

## Step 1: Import Libraries

In [ ]:
import sys
import os
import pandas as pd
from backtesting import Backtest, Strategy
import ta
import sqlite3
import matplotlib.pyplot as plt

# Add the project root directory to sys.path
sys.path.append(os.path.abspath('../..'))

: 

## Step 2: Load Historical Data for XAUUSDm

In [ ]:
# Connect to the database
conn = sqlite3.connect('../../src/data/market_data.sqlite')

# Fetch XAUUSDm data
symbol = 'XAUUSDm'
query = f"SELECT * FROM market_data WHERE symbol='{symbol}' ORDER BY time"
data = pd.read_sql(query, conn)
conn.close()

print(f"Fetched {len(data)} rows of market data for {symbol}")
data.head()

## Step 3: Prepare Data for Backtesting.py

In [ ]:
# Prepare data for Backtesting.py
data['time'] = pd.to_datetime(data['time'])
data.set_index('time', inplace=True)
data = data.sort_index()

# Rename columns to match Backtesting.py requirements
data = data.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'tick_volume': 'Volume'
})

# Ensure all required columns are present
if 'Volume' not in data.columns:
    data['Volume'] = 0

data.head()

## Step 4: Visualize the Data

In [ ]:
# Plot the closing prices
plt.figure(figsize=(12, 6))
plt.plot(data.index, data['Close'], label='Close Price')
plt.title(f'XAUUSDm Close Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

## Step 5: Define the RSI Strategy

In [ ]:
from backtests.strategies.backtest_rsi_strategy import BacktestRSIStrategy

## Step 6: Compute and Plot the RSI Indicator

In [ ]:
# Compute RSI for visualization
rsi = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()

# Plot RSI
plt.figure(figsize=(12, 6))
plt.plot(data.index, rsi, label='RSI', color='purple')
plt.axhline(70, color='red', linestyle='--', label='Overbought (70)')
plt.axhline(30, color='green', linestyle='--', label='Oversold (30)')
plt.title('RSI Indicator for XAUUSDm')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.legend()
plt.show()

## Step 7: Run the Backtest

In [ ]:
# Run the backtest
bt = Backtest(data, BacktestRSIStrategy, cash=10000, commission=0.001)
results = bt.run()

# Display basic stats
print(f"Total Return: {results['Return [%]']:.2f}%")
print(f"Number of Trades: {results['# Trades']}")
print(f"Win Rate: {results['Win Rate [%]']:.2f}%")

## Step 8: Plot the Backtest Results

In [ ]:
# Ensure the results directory exists
results_dir = os.path.abspath('../results')
os.makedirs(results_dir, exist_ok=True)

# Plot the backtest results (interactive HTML plot)
output_file = os.path.join(results_dir, 'equity_curve_rsi_XAUUSDm.html')
bt.plot(filename=output_file, open_browser=False)
print(f"Equity curve saved to {output_file}")